In [2]:
import pandas as pd
import cv2
import numpy as np
from PIL import Image
import pytesseract
import tensorflow as tf
import torch
import torchvision.transforms as transforms
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from src.utils import download_images

# Set up paths
TRAIN_CSV = 'dataset/train.csv'
TEST_CSV = 'dataset/test.csv'
OUTPUT_CSV = 'test_out.csv'
IMAGE_DIR = 'images/'

# Load the dataset
train_df = pd.read_csv()
test_df = pd.read_csv(TEST_CSV)

# Download images
download_images(train_df['image_link'].tolist(), IMAGE_DIR)
download_images(test_df['image_link'].tolist(), IMAGE_DIR)

# Preprocessing
def preprocess_image(image_path):
    image = Image.open(image_path)
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    return transform(image).unsqueeze(0)  # Add batch dimension

# Dummy model function (Replace with your model)
def predict_image(image_tensor):
    # Load a pre-trained model (e.g., ResNet) and run inference
    model = torchvision.models.resnet18(pretrained=True)
    model.eval()
    with torch.no_grad():
        outputs = model(image_tensor)
        # Dummy implementation: return a random prediction
        return {'value': '34', 'unit': 'gram'}

# OCR to extract text
def extract_text_from_image(image_path):
    image = Image.open(image_path)
    text = pytesseract.image_to_string(image)
    return text

# Predictions
def predict_entities(df, model):
    predictions = []
    for index, row in df.iterrows():
        image_path = f"{IMAGE_DIR}/{row['index']}.jpg"
        image_tensor = preprocess_image(image_path)
        prediction = model(image_tensor)
        if prediction:
            formatted_prediction = f"{prediction['value']} {prediction['unit']}"
        else:
            formatted_prediction = ""
        predictions.append({'index': row['index'], 'prediction': formatted_prediction})
    return predictions

# Create output DataFrame
predictions = predict_entities(test_df, predict_image)
output_df = pd.DataFrame(predictions)

# Save the output file
output_df.to_csv(OUTPUT_CSV, index=False)

print("Predictions saved to", OUTPUT_CSV)


2024-09-15 10:40:44.620849: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-15 10:40:46.854420: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-15 10:40:46.857690: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-15 10:40:52.263054: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


ModuleNotFoundError: No module named 'src'

In [5]:
import pandas as pd
import requests
from PIL import Image
import pytesseract
from io import BytesIO
import re

# Load datasets
train_data = pd.read_csv('/home/rguktrkvalley/Desktop/train1.csv')
test_data = pd.read_csv('/home/rguktrkvalley/Desktop/test1.csv')

# Define constants
ALLOWED_UNITS = {'gram', 'centimetre', 'millilitre', 'kilogram', 'millimetre', 'ounce', 'litre'}
# Function to download images
def download_image(url):
    try:
        response = requests.get(url)
        img = Image.open(BytesIO(response.content))
        return img
    except Exception as e:
        print(f"Error downloading image: {e}")
        return None

# Function to perform OCR and extract text from images
def extract_text_from_image(image):
    try:
        text = pytesseract.image_to_string(image)
        return text
    except Exception as e:
        print(f"Error extracting text: {e}")
        return ""

# Function to clean and process extracted text
def clean_extracted_text(text, entity_name):
    # Regex to find numbers followed by units
    pattern = r'(\d+\.?\d*)\s*(gram|g|cm|centimeter|ml|ounce|kg|kilogram|litre|mm)'
    matches = re.findall(pattern, text.lower())  # Find all matches
    
    cleaned_results = []
    
    for match in matches:
        number = match[0]
        unit = match[1]
        # Map shorthand to allowed unit
        unit_mapping = {
            'g': 'gram',
            'cm': 'centimetre',
            'ml': 'millilitre',
            'kg': 'kilogram',
            'mm': 'millimetre',
            'ounce': 'ounce',
            'litre': 'litre'
        }
        unit = unit_mapping.get(unit, unit)
        if unit in ALLOWED_UNITS:
            cleaned_results.append(f"{number} {unit}")
    
    # Ensure that the result aligns with the entity name
    if entity_name == 'item_weight':
        # Example: refine result if entity_name suggests weight
        if cleaned_results:
            return cleaned_results[0]
    elif entity_name == 'item_volume':
        # Example: refine result if entity_name suggests volume
        if cleaned_results:
            return cleaned_results[0]
    # Add more conditions based on entity_name as needed

    return cleaned_results[0] if cleaned_results else ""

# Main processing loop for the test dataset
predictions = []
for idx, row in test_data.iterrows():
    image_url = row['image_link']
    img = download_image(image_url)
    
    if img:
        extracted_text = extract_text_from_image(img)
        cleaned_text = clean_extracted_text(extracted_text,row['entity_name'])
        
        # Placeholder logic: This should be replaced by actual entity extraction logic
        # Currently just outputting the cleaned text
        prediction = cleaned_text
        
        # Append the prediction to the list
        predictions.append({"index": row['index'], "prediction": prediction})
    else:
        predictions.append({"index": row['index'], "prediction": ""})
        

# Convert predictions to DataFrame
predictions_df = pd.DataFrame(predictions)

# Save the predictions to CSV in the required format
output_file = '/home/rguktrkvalley/Desktop/test_out1.csv'
predictions_df.to_csv(output_file, index=False)

print(f"Predictions saved to {output_file}")


Predictions saved to /home/rguktrkvalley/Desktop/test_out1.csv
